In [30]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
import os
from six.moves.urllib.request import urlopen
import numpy as np
import pandas as pd

### Execute the following cell only if you want the logging info supressed for anythign other than error message

In [32]:
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '2'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

### Data set download

In [33]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [34]:
def main():
  # If the training and test sets aren't stored locally, download them.
  if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

  if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

main()

### Load downloaded CSV files as TF data sets

In [35]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [36]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

### Define input function

In [37]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True)

### Define test input function and evaluate accuracy

In [38]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False)


## Hyperparameter study with various activation functions

### Hyperparameters' list

In [59]:
# Hyperparameters' lists
learn_rate = [0.0001*2**(x) for x in range(1,12,2)]
dropout_prob = [0.01, 0.05, 0.1, 0.25]
l1_reg = [0.01, 0.1, 1.0, 10.0]
train_steps=[100, 500, 2500]

import time

### Sigmoid activation

In [60]:
# Loop counts to keep track of whole training run over the multi-dimensional hyperparameter space
loops = len(learn_rate)*len(dropout_prob)*len(train_steps)*len(l1_reg)
loop_remain = loops

# Empty lists to append to
accu_sigmoid_temp = []
time_sigmoid_temp = []

#### Main iteration loop (this will take some time, please sit back and relax, or go get a cuppa!)

In [61]:
from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                               
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,10,5], dropout=d,
                                                      n_classes=3,
                                                    optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l,
                                                    l1_regularization_strength=r),
                                                   activation_fn=tf.nn.sigmoid)

                classifier.train(input_fn=train_input_fn, steps=t)
                accu_sigmoid_temp.append(classifier.evaluate(input_fn=test_input_fn)["accuracy"])
                
                end = time.time()
                
                loop_remain = loop_remain-1
                pct_done= round(100*(loops-loop_remain)/loops,2)
                
                time_sigmoid_temp.append(end-start)
                
                if (loop_remain>0):
                    print("Finished this run. Time taken: {} seconds. {}% done. {} runs remianing."
                          .format(round(end-start,3),pct_done,loop_remain))
                else:
                    t_array = np.array(time_sigmoid_temp)
                    t_total = t_array.sum()
                    print("Time taken for this run: {} seconds. {}% done. Finished the job in total {} seconds."
                          .format(round(end-start,3),pct_done,round(t_total,3)))

Finished this run. Time taken: 7.276 seconds. 0.35% done. 287 runs remianing.
Finished this run. Time taken: 10.801 seconds. 0.69% done. 286 runs remianing.
Finished this run. Time taken: 21.342 seconds. 1.04% done. 285 runs remianing.
Finished this run. Time taken: 6.794 seconds. 1.39% done. 284 runs remianing.
Finished this run. Time taken: 9.219 seconds. 1.74% done. 283 runs remianing.
Finished this run. Time taken: 21.2 seconds. 2.08% done. 282 runs remianing.
Finished this run. Time taken: 6.82 seconds. 2.43% done. 281 runs remianing.
Finished this run. Time taken: 9.565 seconds. 2.78% done. 280 runs remianing.
Finished this run. Time taken: 22.285 seconds. 3.12% done. 279 runs remianing.
Finished this run. Time taken: 7.05 seconds. 3.47% done. 278 runs remianing.
Finished this run. Time taken: 8.504 seconds. 3.82% done. 277 runs remianing.
Finished this run. Time taken: 19.191 seconds. 4.17% done. 276 runs remianing.
Finished this run. Time taken: 6.629 seconds. 4.51% done. 275 r

Finished this run. Time taken: 18.834 seconds. 36.46% done. 183 runs remianing.
Finished this run. Time taken: 6.943 seconds. 36.81% done. 182 runs remianing.
Finished this run. Time taken: 8.892 seconds. 37.15% done. 181 runs remianing.
Finished this run. Time taken: 19.274 seconds. 37.5% done. 180 runs remianing.
Finished this run. Time taken: 6.785 seconds. 37.85% done. 179 runs remianing.
Finished this run. Time taken: 8.794 seconds. 38.19% done. 178 runs remianing.
Finished this run. Time taken: 19.298 seconds. 38.54% done. 177 runs remianing.
Finished this run. Time taken: 6.841 seconds. 38.89% done. 176 runs remianing.
Finished this run. Time taken: 8.827 seconds. 39.24% done. 175 runs remianing.
Finished this run. Time taken: 19.256 seconds. 39.58% done. 174 runs remianing.
Finished this run. Time taken: 6.679 seconds. 39.93% done. 173 runs remianing.
Finished this run. Time taken: 8.728 seconds. 40.28% done. 172 runs remianing.
Finished this run. Time taken: 19.029 seconds. 40

Finished this run. Time taken: 9.038 seconds. 72.57% done. 79 runs remianing.
Finished this run. Time taken: 18.617 seconds. 72.92% done. 78 runs remianing.
Finished this run. Time taken: 6.375 seconds. 73.26% done. 77 runs remianing.
Finished this run. Time taken: 8.603 seconds. 73.61% done. 76 runs remianing.
Finished this run. Time taken: 19.392 seconds. 73.96% done. 75 runs remianing.
Finished this run. Time taken: 6.53 seconds. 74.31% done. 74 runs remianing.
Finished this run. Time taken: 8.805 seconds. 74.65% done. 73 runs remianing.
Finished this run. Time taken: 18.706 seconds. 75.0% done. 72 runs remianing.
Finished this run. Time taken: 6.941 seconds. 75.35% done. 71 runs remianing.
Finished this run. Time taken: 8.557 seconds. 75.69% done. 70 runs remianing.
Finished this run. Time taken: 19.47 seconds. 76.04% done. 69 runs remianing.
Finished this run. Time taken: 6.673 seconds. 76.39% done. 68 runs remianing.
Finished this run. Time taken: 8.889 seconds. 76.74% done. 67 r

#### Make copies of the accuracy and execution times' list using .copy() method (i.e. not just memory referencing). Then reverse the temp lists

In [62]:
accu_sigmoid=accu_sigmoid_temp.copy()
time_sigmoid = time_sigmoid_temp.copy()

accu_sigmoid_temp.reverse()
time_sigmoid_temp.reverse()

#### Run the iteration loop in the exact same order to create the desired dataframe

In [63]:
temp=[]
for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accu_sigmoid_temp.pop(),time_sigmoid_temp.pop()])

df_sigmoid = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])

In [64]:
df_sigmoid.to_csv('Sigmoid_data.csv')
df_sigmoid

,dropout,learning.rate,L1.regularization,training.steps,accuracy,execution.time
0,0.01,0.0002,0.01,100,0.466667,7.275735
1,0.01,0.0002,0.01,500,0.466667,10.801194
2,0.01,0.0002,0.01,2500,0.266667,21.342407
3,0.01,0.0002,0.10,100,0.466667,6.794188
4,0.01,0.0002,0.10,500,0.466667,9.219126
5,0.01,0.0002,0.10,2500,0.266667,21.200007
6,0.01,0.0002,1.00,100,0.466667,6.820385
7,0.01,0.0002,1.00,500,0.266667,9.564777
8,0.01,0.0002,1.00,2500,0.266667,22.285081
9,0.01,0.0002,10.00,100,0.266667,7.049996


### RELU activation function

In [65]:
# Loop counts to keep track of whole training run over the multi-dimensional hyperparameter space
loops = len(learn_rate)*len(dropout_prob)*len(train_steps)*len(l1_reg)
loop_remain = loops

# Empty lists to append to
accu_relu_temp = []
time_relu_temp = []

# Main iteration loop

from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                               
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,10,5], 
                                                        dropout=d,n_classes=3,
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l,
                                                    l1_regularization_strength=r),
                                                   activation_fn=tf.nn.relu)

                classifier.train(input_fn=train_input_fn, steps=t)
                accu_relu_temp.append(classifier.evaluate(input_fn=test_input_fn)["accuracy"])
                
                end = time.time()
                
                loop_remain = loop_remain-1
                pct_done= round(100*(loops-loop_remain)/loops,2)
                
                time_relu_temp.append(end-start)
                
                if (loop_remain>0):
                    print("Finished this run. Time taken: {} seconds. {}% done. {} runs remianing."
                          .format(round(end-start,3),pct_done,loop_remain))
                else:
                    t_array = np.array(time_relu_temp)
                    t_total = t_array.sum()
                    print("Time taken for this run: {} seconds. {}% done. Finished the job in total {} seconds."
                          .format(round(end-start,3),pct_done,round(t_total,3)))

Finished this run. Time taken: 6.58 seconds. 0.35% done. 287 runs remianing.
Finished this run. Time taken: 9.085 seconds. 0.69% done. 286 runs remianing.
Finished this run. Time taken: 19.099 seconds. 1.04% done. 285 runs remianing.
Finished this run. Time taken: 6.999 seconds. 1.39% done. 284 runs remianing.
Finished this run. Time taken: 8.635 seconds. 1.74% done. 283 runs remianing.
Finished this run. Time taken: 18.604 seconds. 2.08% done. 282 runs remianing.
Finished this run. Time taken: 6.755 seconds. 2.43% done. 281 runs remianing.
Finished this run. Time taken: 8.809 seconds. 2.78% done. 280 runs remianing.
Finished this run. Time taken: 19.148 seconds. 3.12% done. 279 runs remianing.
Finished this run. Time taken: 6.798 seconds. 3.47% done. 278 runs remianing.
Finished this run. Time taken: 8.493 seconds. 3.82% done. 277 runs remianing.
Finished this run. Time taken: 19.49 seconds. 4.17% done. 276 runs remianing.
Finished this run. Time taken: 7.389 seconds. 4.51% done. 275 

Finished this run. Time taken: 19.064 seconds. 36.46% done. 183 runs remianing.
Finished this run. Time taken: 6.813 seconds. 36.81% done. 182 runs remianing.
Finished this run. Time taken: 8.91 seconds. 37.15% done. 181 runs remianing.
Finished this run. Time taken: 19.226 seconds. 37.5% done. 180 runs remianing.
Finished this run. Time taken: 6.959 seconds. 37.85% done. 179 runs remianing.
Finished this run. Time taken: 8.967 seconds. 38.19% done. 178 runs remianing.
Finished this run. Time taken: 19.194 seconds. 38.54% done. 177 runs remianing.
Finished this run. Time taken: 6.812 seconds. 38.89% done. 176 runs remianing.
Finished this run. Time taken: 9.239 seconds. 39.24% done. 175 runs remianing.
Finished this run. Time taken: 18.819 seconds. 39.58% done. 174 runs remianing.
Finished this run. Time taken: 6.628 seconds. 39.93% done. 173 runs remianing.
Finished this run. Time taken: 8.819 seconds. 40.28% done. 172 runs remianing.
Finished this run. Time taken: 19.751 seconds. 40.

Finished this run. Time taken: 9.008 seconds. 72.57% done. 79 runs remianing.
Finished this run. Time taken: 19.548 seconds. 72.92% done. 78 runs remianing.
Finished this run. Time taken: 6.31 seconds. 73.26% done. 77 runs remianing.
Finished this run. Time taken: 9.287 seconds. 73.61% done. 76 runs remianing.
Finished this run. Time taken: 19.422 seconds. 73.96% done. 75 runs remianing.
Finished this run. Time taken: 6.612 seconds. 74.31% done. 74 runs remianing.
Finished this run. Time taken: 8.568 seconds. 74.65% done. 73 runs remianing.
Finished this run. Time taken: 18.826 seconds. 75.0% done. 72 runs remianing.
Finished this run. Time taken: 6.605 seconds. 75.35% done. 71 runs remianing.
Finished this run. Time taken: 8.601 seconds. 75.69% done. 70 runs remianing.
Finished this run. Time taken: 18.958 seconds. 76.04% done. 69 runs remianing.
Finished this run. Time taken: 7.0 seconds. 76.39% done. 68 runs remianing.
Finished this run. Time taken: 8.577 seconds. 76.74% done. 67 ru

In [66]:
accu_relu=accu_relu_temp.copy()
time_relu = time_relu_temp.copy()

accu_relu_temp.reverse()
time_relu_temp.reverse()

temp=[]
for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accu_relu_temp.pop(),time_relu_temp.pop()])

df_relu = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])

In [67]:
df_relu.to_csv('RELU_data.csv')
df_relu

,dropout,learning.rate,L1.regularization,training.steps,accuracy,execution.time
0,0.01,0.0002,0.01,100,0.266667,6.579509
1,0.01,0.0002,0.01,500,0.266667,9.084906
2,0.01,0.0002,0.01,2500,0.266667,19.098795
3,0.01,0.0002,0.10,100,0.266667,6.999017
4,0.01,0.0002,0.10,500,0.300000,8.634501
5,0.01,0.0002,0.10,2500,0.466667,18.603963
6,0.01,0.0002,1.00,100,0.266667,6.754703
7,0.01,0.0002,1.00,500,0.266667,8.809044
8,0.01,0.0002,1.00,2500,0.266667,19.147817
9,0.01,0.0002,10.00,100,0.266667,6.797995


## tanh(x) activation function

In [71]:
# Loop counts to keep track of whole training run over the multi-dimensional hyperparameter space
loops = len(learn_rate)*len(dropout_prob)*len(train_steps)*len(l1_reg)
loop_remain = loops

# Empty lists to append to
accu_tanh_temp = []
time_tanh_temp = []

# Main iteration loop

from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                               
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,10,5], 
                                                        dropout=d,n_classes=3,
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l,
                                                    l1_regularization_strength=r),
                                                   activation_fn=tf.nn.tanh)

                classifier.train(input_fn=train_input_fn, steps=t)
                accu_tanh_temp.append(classifier.evaluate(input_fn=test_input_fn)["accuracy"])
                
                end = time.time()
                
                loop_remain = loop_remain-1
                pct_done= round(100*(loops-loop_remain)/loops,2)
                
                time_tanh_temp.append(end-start)
                
                if (loop_remain>0):
                    print("Finished this run. Time taken: {} seconds. {}% done. {} runs remianing."
                          .format(round(end-start,3),pct_done,loop_remain))
                else:
                    t_array = np.array(time_tanh_temp)
                    t_total = t_array.sum()
                    print("Time taken for this run: {} seconds. {}% done. Finished the job in total {} seconds."
                          .format(round(end-start,3),pct_done,round(t_total,3)))

Finished this run. Time taken: 6.709 seconds. 0.35% done. 287 runs remianing.
Finished this run. Time taken: 10.116 seconds. 0.69% done. 286 runs remianing.
Finished this run. Time taken: 20.873 seconds. 1.04% done. 285 runs remianing.
Finished this run. Time taken: 8.113 seconds. 1.39% done. 284 runs remianing.
Finished this run. Time taken: 10.008 seconds. 1.74% done. 283 runs remianing.
Finished this run. Time taken: 21.253 seconds. 2.08% done. 282 runs remianing.
Finished this run. Time taken: 7.503 seconds. 2.43% done. 281 runs remianing.
Finished this run. Time taken: 10.979 seconds. 2.78% done. 280 runs remianing.
Finished this run. Time taken: 23.544 seconds. 3.12% done. 279 runs remianing.
Finished this run. Time taken: 7.821 seconds. 3.47% done. 278 runs remianing.
Finished this run. Time taken: 12.203 seconds. 3.82% done. 277 runs remianing.
Finished this run. Time taken: 21.985 seconds. 4.17% done. 276 runs remianing.
Finished this run. Time taken: 7.262 seconds. 4.51% done

Finished this run. Time taken: 31.992 seconds. 36.46% done. 183 runs remianing.
Finished this run. Time taken: 10.272 seconds. 36.81% done. 182 runs remianing.
Finished this run. Time taken: 11.328 seconds. 37.15% done. 181 runs remianing.
Finished this run. Time taken: 28.259 seconds. 37.5% done. 180 runs remianing.
Finished this run. Time taken: 9.495 seconds. 37.85% done. 179 runs remianing.
Finished this run. Time taken: 10.973 seconds. 38.19% done. 178 runs remianing.
Finished this run. Time taken: 21.039 seconds. 38.54% done. 177 runs remianing.
Finished this run. Time taken: 7.66 seconds. 38.89% done. 176 runs remianing.
Finished this run. Time taken: 9.655 seconds. 39.24% done. 175 runs remianing.
Finished this run. Time taken: 23.703 seconds. 39.58% done. 174 runs remianing.
Finished this run. Time taken: 14.863 seconds. 39.93% done. 173 runs remianing.
Finished this run. Time taken: 14.447 seconds. 40.28% done. 172 runs remianing.
Finished this run. Time taken: 41.172 seconds

Finished this run. Time taken: 9.582 seconds. 72.57% done. 79 runs remianing.
Finished this run. Time taken: 20.803 seconds. 72.92% done. 78 runs remianing.
Finished this run. Time taken: 7.195 seconds. 73.26% done. 77 runs remianing.
Finished this run. Time taken: 9.074 seconds. 73.61% done. 76 runs remianing.
Finished this run. Time taken: 20.534 seconds. 73.96% done. 75 runs remianing.
Finished this run. Time taken: 7.555 seconds. 74.31% done. 74 runs remianing.
Finished this run. Time taken: 8.931 seconds. 74.65% done. 73 runs remianing.
Finished this run. Time taken: 20.22 seconds. 75.0% done. 72 runs remianing.
Finished this run. Time taken: 7.256 seconds. 75.35% done. 71 runs remianing.
Finished this run. Time taken: 8.712 seconds. 75.69% done. 70 runs remianing.
Finished this run. Time taken: 19.5 seconds. 76.04% done. 69 runs remianing.
Finished this run. Time taken: 7.234 seconds. 76.39% done. 68 runs remianing.
Finished this run. Time taken: 9.124 seconds. 76.74% done. 67 ru

In [72]:
accu_tanh=accu_tanh_temp.copy()
time_tanh = time_tanh_temp.copy()

accu_tanh_temp.reverse()
time_tanh_temp.reverse()

temp=[]
for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accu_tanh_temp.pop(),time_tanh_temp.pop()])

df_tanh = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])

In [73]:
df_tanh.to_csv('Tanh_data.csv')
df_tanh

,dropout,learning.rate,L1.regularization,training.steps,accuracy,execution.time
0,0.01,0.0002,0.01,100,0.466667,6.709321
1,0.01,0.0002,0.01,500,0.566667,10.116145
2,0.01,0.0002,0.01,2500,0.466667,20.873255
3,0.01,0.0002,0.10,100,0.733333,8.113106
4,0.01,0.0002,0.10,500,0.000000,10.007829
5,0.01,0.0002,0.10,2500,0.433333,21.253207
6,0.01,0.0002,1.00,100,0.466667,7.503400
7,0.01,0.0002,1.00,500,0.200000,10.979250
8,0.01,0.0002,1.00,2500,0.266667,23.544298
9,0.01,0.0002,10.00,100,0.266667,7.820700


## Exponential linear unit (ELU) activation function
#### Check the following paper: https://arxiv.org/abs/1511.07289

In [74]:
# Loop counts to keep track of whole training run over the multi-dimensional hyperparameter space
loops = len(learn_rate)*len(dropout_prob)*len(train_steps)*len(l1_reg)
loop_remain = loops

# Empty lists to append to
accu_elu_temp = []
time_elu_temp = []

# Main iteration loop

from tensorflow import logging
logging.set_verbosity(logging.ERROR)

for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                
                start = time.time()
                               
                classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,hidden_units=[5,10,5], 
                                                        dropout=d,n_classes=3,
                                                        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=l,
                                                    l1_regularization_strength=r),
                                                   activation_fn=tf.nn.elu)

                classifier.train(input_fn=train_input_fn, steps=t)
                accu_elu_temp.append(classifier.evaluate(input_fn=test_input_fn)["accuracy"])
                
                end = time.time()
                
                loop_remain = loop_remain-1
                pct_done= round(100*(loops-loop_remain)/loops,2)
                
                time_elu_temp.append(end-start)
                
                if (loop_remain>0):
                    print("Finished this run. Time taken: {} seconds. {}% done. {} runs remianing."
                          .format(round(end-start,3),pct_done,loop_remain))
                else:
                    t_array = np.array(time_elu_temp)
                    t_total = t_array.sum()
                    print("Time taken for this run: {} seconds. {}% done. Finished the job in total {} seconds."
                          .format(round(end-start,3),pct_done,round(t_total,3)))

Finished this run. Time taken: 6.692 seconds. 0.35% done. 287 runs remianing.
Finished this run. Time taken: 9.476 seconds. 0.69% done. 286 runs remianing.
Finished this run. Time taken: 20.094 seconds. 1.04% done. 285 runs remianing.
Finished this run. Time taken: 7.329 seconds. 1.39% done. 284 runs remianing.
Finished this run. Time taken: 10.551 seconds. 1.74% done. 283 runs remianing.
Finished this run. Time taken: 20.757 seconds. 2.08% done. 282 runs remianing.
Finished this run. Time taken: 7.42 seconds. 2.43% done. 281 runs remianing.
Finished this run. Time taken: 9.078 seconds. 2.78% done. 280 runs remianing.
Finished this run. Time taken: 20.955 seconds. 3.12% done. 279 runs remianing.
Finished this run. Time taken: 7.097 seconds. 3.47% done. 278 runs remianing.
Finished this run. Time taken: 9.075 seconds. 3.82% done. 277 runs remianing.
Finished this run. Time taken: 19.943 seconds. 4.17% done. 276 runs remianing.
Finished this run. Time taken: 7.374 seconds. 4.51% done. 27

Finished this run. Time taken: 22.175 seconds. 36.46% done. 183 runs remianing.
Finished this run. Time taken: 7.731 seconds. 36.81% done. 182 runs remianing.
Finished this run. Time taken: 9.558 seconds. 37.15% done. 181 runs remianing.
Finished this run. Time taken: 21.221 seconds. 37.5% done. 180 runs remianing.
Finished this run. Time taken: 7.477 seconds. 37.85% done. 179 runs remianing.
Finished this run. Time taken: 10.567 seconds. 38.19% done. 178 runs remianing.
Finished this run. Time taken: 21.136 seconds. 38.54% done. 177 runs remianing.
Finished this run. Time taken: 7.93 seconds. 38.89% done. 176 runs remianing.
Finished this run. Time taken: 10.187 seconds. 39.24% done. 175 runs remianing.
Finished this run. Time taken: 20.874 seconds. 39.58% done. 174 runs remianing.
Finished this run. Time taken: 7.453 seconds. 39.93% done. 173 runs remianing.
Finished this run. Time taken: 10.284 seconds. 40.28% done. 172 runs remianing.
Finished this run. Time taken: 20.423 seconds. 

Finished this run. Time taken: 9.555 seconds. 72.57% done. 79 runs remianing.
Finished this run. Time taken: 21.32 seconds. 72.92% done. 78 runs remianing.
Finished this run. Time taken: 7.847 seconds. 73.26% done. 77 runs remianing.
Finished this run. Time taken: 9.452 seconds. 73.61% done. 76 runs remianing.
Finished this run. Time taken: 21.266 seconds. 73.96% done. 75 runs remianing.
Finished this run. Time taken: 7.526 seconds. 74.31% done. 74 runs remianing.
Finished this run. Time taken: 10.205 seconds. 74.65% done. 73 runs remianing.
Finished this run. Time taken: 21.17 seconds. 75.0% done. 72 runs remianing.
Finished this run. Time taken: 8.122 seconds. 75.35% done. 71 runs remianing.
Finished this run. Time taken: 10.143 seconds. 75.69% done. 70 runs remianing.
Finished this run. Time taken: 21.379 seconds. 76.04% done. 69 runs remianing.
Finished this run. Time taken: 6.81 seconds. 76.39% done. 68 runs remianing.
Finished this run. Time taken: 9.091 seconds. 76.74% done. 67 

In [75]:
accu_elu=accu_elu_temp.copy()
time_elu = time_elu_temp.copy()

accu_elu_temp.reverse()
time_elu_temp.reverse()

temp=[]
for d in dropout_prob:
    for l in learn_rate:
        for r in l1_reg:
            for t in train_steps:
                temp.append([d,l,r,t,accu_elu_temp.pop(),time_elu_temp.pop()])

df_elu = pd.DataFrame(data=temp, columns=['dropout', 'learning.rate','L1.regularization',
                                      'training.steps','accuracy','execution.time'])

In [76]:
df_elu.to_csv('ELU_data.csv')
df_elu

,dropout,learning.rate,L1.regularization,training.steps,accuracy,execution.time
0,0.01,0.0002,0.01,100,0.266667,6.691806
1,0.01,0.0002,0.01,500,0.266667,9.475716
2,0.01,0.0002,0.01,2500,0.500000,20.093862
3,0.01,0.0002,0.10,100,0.466667,7.328618
4,0.01,0.0002,0.10,500,0.266667,10.550757
5,0.01,0.0002,0.10,2500,0.266667,20.756750
6,0.01,0.0002,1.00,100,0.733333,7.420426
7,0.01,0.0002,1.00,500,0.266667,9.078148
8,0.01,0.0002,1.00,2500,0.266667,20.954795
9,0.01,0.0002,10.00,100,0.033333,7.096574


## Comparison of activation functions

In [77]:
print("> 80% Accuracy with Exponential Linear Unit activation:",df_elu[df_elu['accuracy']>0.8].count()['dropout'])
print("> 80% Accuracy with Rectified Linear Unit activation:",df_relu[df_relu['accuracy']>0.8].count()['dropout'])
print("> 80% Accuracy with tanh(x) activation:",df_tanh[df_tanh['accuracy']>0.8].count()['dropout'])
print("> 80% Accuracy with sigmoid function activation:",df_sigmoid[df_sigmoid['accuracy']>0.8].count()['dropout'])

> 80% Accuracy with Exponential Linear Unit activation: 111
> 80% Accuracy with Rectified Linear Unit activation: 75
> 80% Accuracy with tanh(x) activation: 105
> 80% Accuracy with sigmoid function activation: 33


In [81]:
df_sigmoid.to_csv('Sigmoid_data.csv')
df_relu.to_csv('RELU_data.csv')
df_tanh.to_csv('Tanh_data.csv')
df_elu.to_csv('ELU_data.csv')